In [7]:
#Imports
import numpy as np
from midiutil import MIDIFile
import matplotlib.pyplot as plt
import ternary
from ternary.helpers import simplex_iterator
import graphviz
import anytree as at
from anytree import Node, RenderTree
from anytree.exporter import DotExporter
import theano.tensor as T
from theano import function

# helpers
import sys
sys.path.insert(0, 'helpers/')
from tree_parser import TreeParser
from midi import *

import os
os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/Graphviz2.38/bin/'

WARNING (theano.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
C:\Users\Spouep\Anaconda3\lib\site-packages\theano\configdefaults.py:560: UserWarning: DeprecationWarning: there is no c++ compiler.This is deprecated and with Theano 0.11 a c++ compiler will be mandatory
  warnings.warn("DeprecationWarning: there is no c++ compiler."
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [24]:
# CONSTANTS
L = 16 # length of measure
N = int(np.log2(L)+1) # Number of levels (including 0)

In [8]:
# Initial THEANO symbolic variables

p1 = T.dscalar('p1')
p2 = T.dscalar('p2')
p3 = T.dscalar('p3')

# Set of all trees

In [15]:
# Templates level 1
templates_1 = set()
templates_1.add(p2)
templates_1.add(p3)

# Templates level 2
templates_2 = set()
templates_2.add(p2)
templates_2.add(p3)

for elem1 in templates_1:
    for elem2 in templates_1:
        templates_2.add((elem1, elem2))
        
# Templates level 3
templates_3 = set()
templates_3.add(p2)
templates_3.add(p3)

for elem1 in templates_2:
    for elem2 in templates_2:
        templates_3.add((elem1, elem2))
        
# Templates level 4
templates_4 = set()
templates_4.add(p2)
templates_4.add(p3)

for elem1 in templates_3:
    for elem2 in templates_3:
        templates_4.add((elem1, elem2))
        
# Templates level 5
all_trees = set()
all_trees.add(p2)
all_trees.add(p3)

for elem1 in templates_4:
    for elem2 in templates_4:
        all_trees.add((elem1, elem2))
        
#save
import pickle
with open('files/all_trees.pickle', 'wb') as f:
    pickle.dump(all_trees, f)
    
len(all_trees)

2090918

# Set of all rhythms

In [33]:
all_rhythms = set()

for i in range(2**16): # populating
    all_rhythms.add( tuple(("{0:0%sb}" %L).format(i)) )
    
#save
import pickle
with open('files/all_rhythms.pickle', 'wb') as f:
    pickle.dump(all_rhythms, f)
    
len(all_rhythms)

65536

# 2-way dictionnary between sets of trees and rhythms

In [77]:
# 2-ways dictionary, not too slow...
class bidict(dict):
    def __init__(self, *args, **kwargs):
        super(bidict, self).__init__(*args, **kwargs)
        self.inverse = {}
        for key, value in self.items():
            self.inverse.setdefault(value,[]).append(key) 

    def __setitem__(self, key, value):
        if key in self:
            self.inverse[self[key]].remove(key) 
        super(bidict, self).__setitem__(key, value)
        self.inverse.setdefault(value,[]).append(key)        

    def __delitem__(self, key):
        self.inverse.setdefault(self[key],[]).remove(key)
        if self[key] in self.inverse and not self.inverse[self[key]]: 
            del self.inverse[self[key]]
        super(bidict, self).__delitem__(key)

In [86]:
def tree_to_rhythm(tree, size=L):    
    if(p2 == tree):
        tup = ('1',) + ('0',) * int(size-1)
        return tup    
    elif(p3 == tree):
        tup = ('0',) * int(size)
        return tup    
    else:
        return tree_to_rhythm(tree[0], size/2) + tree_to_rhythm(tree[1], size/2)
    
tree2rhythm = bidict()

for tree in all_trees:
    tree2rhythm[tree] = tree_to_rhythm(tree)
    
#save
import pickle
with open('files/tree2rhythm.pickle', 'wb') as f:
    pickle.dump(tree2rhythm, f)